In [ ]:
#imports:
import pandas as pd
import numpy as np
import math
from time import time

from sklearn.utils import shuffle

import xgboost

from scipy.stats import randint as sp_randint # int distribution for random search
from scipy.stats import uniform # float distribution for random search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive 
drive.mount('/mntDrive') 
data=pd.read_csv("/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/input/prepared_facebook_data.csv")

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# random search
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

# build a classifier
clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)

# specify parameters and distributions
param_dist = {"max_depth": sp_randint(1, 10),
              "min_child_weight": sp_randint(1, 10),
              "alpha": uniform(loc=0, scale=1),
              "lambda": uniform(loc=0, scale=1),
              "subsample":uniform(loc=0.5, scale=0.4),
              "colsample_bytree":uniform(loc=0.5, scale=0.4)}

n_iter_search = 729
#res=pd.DataFrame()


In [ ]:
# run randomized search
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                      n_iter=n_iter_search, cv=5, scoring="r2",iid=False)

start = time()
random_search.fit(X, y)
work_t=time() - start
print("RandomizedSearchCV took ", str(work_t))

res_rand=pd.DataFrame(random_search.cv_results_)
res_rand['run_number']=1
#res=pd.concat([res, res_rand], sort=False)
res_rand['experiment_name']="random search"
res_rand.to_csv("res_rand_1.csv", index=False)

In [ ]:
cp res_rand_1.csv "/mntDrive/My Drive/НИР магистратура/Оптимизация гиперпараметров алгоритмов градиентного бустинга/Практика/data/results/xgboost_perfomance_random_search_GPU_1.csv"